In [ ]:
import os
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from PIL import Image
import timm
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

class ImageClassificationPipeline:
    def __init__(self, data_dir, num_classes=2, batch_size=32, num_epochs=10):
        self.data_dir = data_dir
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        
        # Data transforms
        self.train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        self.val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        self.setup_data_loaders()
        
    def setup_data_loaders(self):
        """Setup train and validation data loaders"""
        train_dataset = datasets.ImageFolder(
            root=os.path.join(self.data_dir, 'train'),
            transform=self.train_transform
        )
        
        val_dataset = datasets.ImageFolder(
            root=os.path.join(self.data_dir, 'valid'),
            transform=self.val_transform
        )
        
        self.train_loader = DataLoader(
            train_dataset, batch_size=self.batch_size, 
            shuffle=True, num_workers=4
        )
        
        self.val_loader = DataLoader(
            val_dataset, batch_size=self.batch_size, 
            shuffle=False, num_workers=4
        )
        
        self.class_names = train_dataset.classes
        print(f"Classes: {self.class_names}")
        print(f"Train samples: {len(train_dataset)}")
        print(f"Val samples: {len(val_dataset)}")

class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        # Freeze early layers
        for param in list(self.resnet.parameters())[:-20]:
            param.requires_grad = False
        
        # Replace final layer
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)

class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetClassifier, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True)
        # Freeze early layers
        for param in list(self.efficientnet.parameters())[:-30]:
            param.requires_grad = False
            
        # Replace classifier
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(self.efficientnet.classifier.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.efficientnet(x)

class YOLOClassifier:
    def __init__(self, data_dir, num_classes):
        self.data_dir = data_dir
        self.num_classes = num_classes
        self.model = None
        
    def prepare_yolo_data(self):
        """Convert folder structure to YOLO format"""
        # This is a simplified version - in practice you'd need proper YOLO dataset structure
        print("Preparing YOLO data structure...")
        # For demonstration, we'll use YOLOv8 classification mode
        
    def train(self, epochs=50):
        """Train YOLO model"""
        # Initialize YOLOv8 classification model
        self.model = YOLO('yolov8n-cls.pt')  # nano version for speed
        
        # Train the model
        results = self.model.train(
            data=self.data_dir,
            epochs=epochs,
            imgsz=224,
            batch=16,
            device=0 if torch.cuda.is_available() else 'cpu'
        )
        return results
    
    def save_model(self, path):
        """Save trained model"""
        if self.model:
            self.model.save(path)
    
    def load_model(self, path):
        """Load saved model"""
        self.model = YOLO(path)
    
    def predict(self, image_path):
        """Run inference"""
        if self.model:
            results = self.model(image_path)
            return results

def train_pytorch_model(model, train_loader, val_loader, num_epochs, device, model_name):
    """Generic training function for PyTorch models"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    
    model = model.to(device)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0
    
    print(f"\nTraining {model_name}...")
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if batch_idx % 20 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                val_loss += criterion(output, target).item()
                
                pred = output.argmax(dim=1)
                val_predictions.extend(pred.cpu().numpy())
                val_targets.extend(target.cpu().numpy())
        
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        val_acc = accuracy_score(val_targets, val_predictions)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)
        
        scheduler.step(avg_val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}')
        print(f'  Val Accuracy: {val_acc:.4f}')
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'best_{model_name.lower()}_model.pth')
            print(f'  New best model saved! Accuracy: {best_val_acc:.4f}')
        
        print('-' * 50)
    
    return model, train_losses, val_losses, val_accuracies

def benchmark_inference_speed(model, test_image_path, device, num_runs=100):
    """Benchmark inference speed"""
    model.eval()
    model = model.to(device)
    
    # Load and preprocess test image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(test_image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    # Warm up
    with torch.no_grad():
        for _ in range(10):
            _ = model(input_tensor)
    
    # Benchmark
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start_time = time.time()
            _ = model(input_tensor)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            end_time = time.time()
            times.append((end_time - start_time) * 1000)  # Convert to milliseconds
    
    avg_time = np.mean(times)
    std_time = np.std(times)
    
    return avg_time, std_time

def create_sample_data():
    """Create sample data structure for testing"""
    import os
    from PIL import Image
    import numpy as np
    
    # Create directory structure
    os.makedirs("sample_data/train/folderA", exist_ok=True)
    os.makedirs("sample_data/train/folderB", exist_ok=True)
    os.makedirs("sample_data/valid/folderA", exist_ok=True)
    os.makedirs("sample_data/valid/folderB", exist_ok=True)
    
    # Generate sample images
    for split in ["train", "valid"]:
        for folder in ["folderA", "folderB"]:
            num_images = 50 if split == "train" else 20
            for i in range(num_images):
                # Generate random image
                if folder == "folderA":
                    # Create images with more red
                    img_array = np.random.randint(100, 255, (224, 224, 3), dtype=np.uint8)
                    img_array[:, :, 0] = np.random.randint(150, 255, (224, 224))  # More red
                else:
                    # Create images with more blue
                    img_array = np.random.randint(100, 255, (224, 224, 3), dtype=np.uint8)
                    img_array[:, :, 2] = np.random.randint(150, 255, (224, 224))  # More blue
                
                img = Image.fromarray(img_array)
                img.save(f"sample_data/{split}/{folder}/image_{i:03d}.jpg")
    
    print("Sample data created in 'sample_data' directory")
    return "sample_data"

def main():
    # Check if data directory exists, if not create sample data
    DATA_DIR = "sample_data"
    if not os.path.exists(DATA_DIR):
        print("Data directory not found. Creating sample data...")
        DATA_DIR = create_sample_data()
    
    # Configuration
    NUM_CLASSES = 2  # folderA and folderB
    BATCH_SIZE = 32
    NUM_EPOCHS = 10  # Reduced for faster training with sample data
    
    # Initialize pipeline
    pipeline = ImageClassificationPipeline(
        data_dir=DATA_DIR,
        num_classes=NUM_CLASSES,
        batch_size=BATCH_SIZE,
        num_epochs=NUM_EPOCHS
    )
    
    device = pipeline.device
    
    # Model 1: ResNet50
    print("="*60)
    print("TRAINING MODEL 1: ResNet50")
    print("="*60)
    
    resnet_model = ResNetClassifier(NUM_CLASSES)
    resnet_model, resnet_train_losses, resnet_val_losses, resnet_val_accs = train_pytorch_model(
        resnet_model, pipeline.train_loader, pipeline.val_loader, 
        NUM_EPOCHS, device, "ResNet50"
    )
    
    # Model 2: EfficientNet
    print("="*60)
    print("TRAINING MODEL 2: EfficientNet-B0")
    print("="*60)
    
    efficientnet_model = EfficientNetClassifier(NUM_CLASSES)
    efficientnet_model, eff_train_losses, eff_val_losses, eff_val_accs = train_pytorch_model(
        efficientnet_model, pipeline.train_loader, pipeline.val_loader, 
        NUM_EPOCHS, device, "EfficientNet"
    )
    
    # Model 3: YOLO (for classification)
    print("="*60)
    print("TRAINING MODEL 3: YOLOv8 Classification")
    print("="*60)
    
    yolo_classifier = YOLOClassifier(DATA_DIR, NUM_CLASSES)
    yolo_results = yolo_classifier.train(epochs=30)
    yolo_classifier.save_model('best_yolo_classifier.pt')
    
    # Save all models
    torch.save(resnet_model.state_dict(), 'resnet50_classifier.pth')
    torch.save(efficientnet_model.state_dict(), 'efficientnet_classifier.pth')
    
    print("="*60)
    print("INFERENCE SPEED BENCHMARKING")
    print("="*60)
    
    # Test image path - use first validation image
    test_image_path = os.path.join(DATA_DIR, "valid", "folderA", "image_000.jpg")
    
    if os.path.exists(test_image_path):
        # Benchmark ResNet
        resnet_avg_time, resnet_std = benchmark_inference_speed(
            resnet_model, test_image_path, device
        )
        print(f"ResNet50 - Average inference time: {resnet_avg_time:.2f}ms ± {resnet_std:.2f}ms")
        
        # Benchmark EfficientNet
        eff_avg_time, eff_std = benchmark_inference_speed(
            efficientnet_model, test_image_path, device
        )
        print(f"EfficientNet - Average inference time: {eff_avg_time:.2f}ms ± {eff_std:.2f}ms")
        
        # Benchmark YOLO
        yolo_times = []
        for _ in range(100):
            start_time = time.time()
            _ = yolo_classifier.predict(test_image_path)
            end_time = time.time()
            yolo_times.append((end_time - start_time) * 1000)
        
        yolo_avg_time = np.mean(yolo_times)
        yolo_std = np.std(yolo_times)
        print(f"YOLO - Average inference time: {yolo_avg_time:.2f}ms ± {yolo_std:.2f}ms")
        
        # Summary
        print("\n" + "="*60)
        print("SUMMARY")
        print("="*60)
        print(f"ResNet50: {resnet_avg_time:.2f}ms - {'✓' if resnet_avg_time < 20 else '✗'} (<20ms)")
        print(f"EfficientNet: {eff_avg_time:.2f}ms - {'✓' if eff_avg_time < 20 else '✗'} (<20ms)")
        print(f"YOLO: {yolo_avg_time:.2f}ms - {'✓' if yolo_avg_time < 20 else '✗'} (<20ms)")
        
        # Performance summary
        results_summary = {
            "models": {
                "resnet50": {
                    "best_val_accuracy": max(resnet_val_accs),
                    "avg_inference_time_ms": resnet_avg_time,
                    "model_path": "resnet50_classifier.pth"
                },
                "efficientnet": {
                    "best_val_accuracy": max(eff_val_accs),
                    "avg_inference_time_ms": eff_avg_time,
                    "model_path": "efficientnet_classifier.pth"
                },
                "yolo": {
                    "avg_inference_time_ms": yolo_avg_time,
                    "model_path": "best_yolo_classifier.pt"
                }
            }
        }
        
        with open('model_performance_summary.json', 'w') as f:
            json.dump(results_summary, f, indent=2)
        
        print("\nResults saved to 'model_performance_summary.json'")
    else:
        print(f"Test image not found at {test_image_path}")
        print("Please update the test_image_path variable with a valid image path")

def load_and_inference_example():
    """Example of loading saved models and running inference"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load ResNet model
    resnet_model = ResNetClassifier(num_classes=2)
    resnet_model.load_state_dict(torch.load('resnet50_classifier.pth', map_location=device))
    resnet_model.eval()
    
    # Load EfficientNet model
    efficientnet_model = EfficientNetClassifier(num_classes=2)
    efficientnet_model.load_state_dict(torch.load('efficientnet_classifier.pth', map_location=device))
    efficientnet_model.eval()
    
    # Load YOLO model
    yolo_model = YOLO('best_yolo_classifier.pt')
    
    # Example inference
    test_image_path = "path/to/test/image.jpg"
    
    if os.path.exists(test_image_path):
        # Preprocess image for PyTorch models
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        image = Image.open(test_image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # ResNet inference
        with torch.no_grad():
            resnet_output = resnet_model(input_tensor)
            resnet_pred = torch.softmax(resnet_output, dim=1)
        
        # EfficientNet inference
        with torch.no_grad():
            eff_output = efficientnet_model(input_tensor)
            eff_pred = torch.softmax(eff_output, dim=1)
        
        # YOLO inference
        yolo_results = yolo_model(test_image_path)
        
        print("Inference Results:")
        print(f"ResNet50 predictions: {resnet_pred}")
        print(f"EfficientNet predictions: {eff_pred}")
        print(f"YOLO results: {yolo_results}")

if __name__ == "__main__":
    main()
    # Uncomment to run inference example
    # load_and_inference_eximport os
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from PIL import Image
import timm
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

class ImageClassificationPipeline:
    def __init__(self, data_dir, num_classes=2, batch_size=32, num_epochs=10):
        self.data_dir = data_dir
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        
        # Data transforms
        self.train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        self.val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        self.setup_data_loaders()
        
    def setup_data_loaders(self):
        """Setup train and validation data loaders"""
        train_dataset = datasets.ImageFolder(
            root=os.path.join(self.data_dir, 'train'),
            transform=self.train_transform
        )
        
        val_dataset = datasets.ImageFolder(
            root=os.path.join(self.data_dir, 'valid'),
            transform=self.val_transform
        )
        
        self.train_loader = DataLoader(
            train_dataset, batch_size=self.batch_size, 
            shuffle=True, num_workers=4
        )
        
        self.val_loader = DataLoader(
            val_dataset, batch_size=self.batch_size, 
            shuffle=False, num_workers=4
        )
        
        self.class_names = train_dataset.classes
        print(f"Classes: {self.class_names}")
        print(f"Train samples: {len(train_dataset)}")
        print(f"Val samples: {len(val_dataset)}")

class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        # Freeze early layers
        for param in list(self.resnet.parameters())[:-20]:
            param.requires_grad = False
        
        # Replace final layer
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)

class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetClassifier, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True)
        # Freeze early layers
        for param in list(self.efficientnet.parameters())[:-30]:
            param.requires_grad = False
            
        # Replace classifier
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(self.efficientnet.classifier.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.efficientnet(x)

class YOLOClassifier:
    def __init__(self, data_dir, num_classes):
        self.data_dir = data_dir
        self.num_classes = num_classes
        self.model = None
        
    def prepare_yolo_data(self):
        """Convert folder structure to YOLO format"""
        # This is a simplified version - in practice you'd need proper YOLO dataset structure
        print("Preparing YOLO data structure...")
        # For demonstration, we'll use YOLOv8 classification mode
        
    def train(self, epochs=50):
        """Train YOLO model"""
        # Initialize YOLOv8 classification model
        self.model = YOLO('yolov8n-cls.pt')  # nano version for speed
        
        # Train the model
        results = self.model.train(
            data=self.data_dir,
            epochs=epochs,
            imgsz=224,
            batch=16,
            device=0 if torch.cuda.is_available() else 'cpu'
        )
        return results
    
    def save_model(self, path):
        """Save trained model"""
        if self.model:
            self.model.save(path)
    
    def load_model(self, path):
        """Load saved model"""
        self.model = YOLO(path)
    
    def predict(self, image_path):
        """Run inference"""
        if self.model:
            results = self.model(image_path)
            return results

def train_pytorch_model(model, train_loader, val_loader, num_epochs, device, model_name):
    """Generic training function for PyTorch models"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    
    model = model.to(device)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0
    
    print(f"\nTraining {model_name}...")
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if batch_idx % 20 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                val_loss += criterion(output, target).item()
                
                pred = output.argmax(dim=1)
                val_predictions.extend(pred.cpu().numpy())
                val_targets.extend(target.cpu().numpy())
        
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        val_acc = accuracy_score(val_targets, val_predictions)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)
        
        scheduler.step(avg_val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}')
        print(f'  Val Accuracy: {val_acc:.4f}')
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'best_{model_name.lower()}_model.pth')
            print(f'  New best model saved! Accuracy: {best_val_acc:.4f}')
        
        print('-' * 50)
    
    return model, train_losses, val_losses, val_accuracies

def benchmark_inference_speed(model, test_image_path, device, num_runs=100):
    """Benchmark inference speed"""
    model.eval()
    model = model.to(device)
    
    # Load and preprocess test image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(test_image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    # Warm up
    with torch.no_grad():
        for _ in range(10):
            _ = model(input_tensor)
    
    # Benchmark
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start_time = time.time()
            _ = model(input_tensor)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            end_time = time.time()
            times.append((end_time - start_time) * 1000)  # Convert to milliseconds
    
    avg_time = np.mean(times)
    std_time = np.std(times)
    
    return avg_time, std_time

def main():
    # Configuration
    DATA_DIR = "path/to/your/data"  # Update this path
    NUM_CLASSES = 2  # folderA and folderB
    BATCH_SIZE = 32
    NUM_EPOCHS = 15
    
    # Initialize pipeline
    pipeline = ImageClassificationPipeline(
        data_dir=DATA_DIR,
        num_classes=NUM_CLASSES,
        batch_size=BATCH_SIZE,
        num_epochs=NUM_EPOCHS
    )
    
    device = pipeline.device
    
    # Model 1: ResNet50
    print("="*60)
    print("TRAINING MODEL 1: ResNet50")
    print("="*60)
    
    resnet_model = ResNetClassifier(NUM_CLASSES)
    resnet_model, resnet_train_losses, resnet_val_losses, resnet_val_accs = train_pytorch_model(
        resnet_model, pipeline.train_loader, pipeline.val_loader, 
        NUM_EPOCHS, device, "ResNet50"
    )
    
    # Model 2: EfficientNet
    print("="*60)
    print("TRAINING MODEL 2: EfficientNet-B0")
    print("="*60)
    
    efficientnet_model = EfficientNetClassifier(NUM_CLASSES)
    efficientnet_model, eff_train_losses, eff_val_losses, eff_val_accs = train_pytorch_model(
        efficientnet_model, pipeline.train_loader, pipeline.val_loader, 
        NUM_EPOCHS, device, "EfficientNet"
    )
    
    # Model 3: YOLO (for classification)
    print("="*60)
    print("TRAINING MODEL 3: YOLOv8 Classification")
    print("="*60)
    
    yolo_classifier = YOLOClassifier(DATA_DIR, NUM_CLASSES)
    yolo_results = yolo_classifier.train(epochs=30)
    yolo_classifier.save_model('best_yolo_classifier.pt')
    
    # Save all models
    torch.save(resnet_model.state_dict(), 'resnet50_classifier.pth')
    torch.save(efficientnet_model.state_dict(), 'efficientnet_classifier.pth')
    
    print("="*60)
    print("INFERENCE SPEED BENCHMARKING")
    print("="*60)
    
    # Test image path (update this)
    test_image_path = "path/to/test/image.jpg"
    
    if os.path.exists(test_image_path):
        # Benchmark ResNet
        resnet_avg_time, resnet_std = benchmark_inference_speed(
            resnet_model, test_image_path, device
        )
        print(f"ResNet50 - Average inference time: {resnet_avg_time:.2f}ms ± {resnet_std:.2f}ms")
        
        # Benchmark EfficientNet
        eff_avg_time, eff_std = benchmark_inference_speed(
            efficientnet_model, test_image_path, device
        )
        print(f"EfficientNet - Average inference time: {eff_avg_time:.2f}ms ± {eff_std:.2f}ms")
        
        # Benchmark YOLO
        yolo_times = []
        for _ in range(100):
            start_time = time.time()
            _ = yolo_classifier.predict(test_image_path)
            end_time = time.time()
            yolo_times.append((end_time - start_time) * 1000)
        
        yolo_avg_time = np.mean(yolo_times)
        yolo_std = np.std(yolo_times)
        print(f"YOLO - Average inference time: {yolo_avg_time:.2f}ms ± {yolo_std:.2f}ms")
        
        # Summary
        print("\n" + "="*60)
        print("SUMMARY")
        print("="*60)
        print(f"ResNet50: {resnet_avg_time:.2f}ms - {'✓' if resnet_avg_time < 20 else '✗'} (<20ms)")
        print(f"EfficientNet: {eff_avg_time:.2f}ms - {'✓' if eff_avg_time < 20 else '✗'} (<20ms)")
        print(f"YOLO: {yolo_avg_time:.2f}ms - {'✓' if yolo_avg_time < 20 else '✗'} (<20ms)")
        
        # Performance summary
        results_summary = {
            "models": {
                "resnet50": {
                    "best_val_accuracy": max(resnet_val_accs),
                    "avg_inference_time_ms": resnet_avg_time,
                    "model_path": "resnet50_classifier.pth"
                },
                "efficientnet": {
                    "best_val_accuracy": max(eff_val_accs),
                    "avg_inference_time_ms": eff_avg_time,
                    "model_path": "efficientnet_classifier.pth"
                },
                "yolo": {
                    "avg_inference_time_ms": yolo_avg_time,
                    "model_path": "best_yolo_classifier.pt"
                }
            }
        }
        
        with open('model_performance_summary.json', 'w') as f:
            json.dump(results_summary, f, indent=2)
        
        print("\nResults saved to 'model_performance_summary.json'")
    else:
        print(f"Test image not found at {test_image_path}")
        print("Please update the test_image_path variable with a valid image path")

def load_and_inference_example():
    """Example of loading saved models and running inference"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load ResNet model
    resnet_model = ResNetClassifier(num_classes=2)
    resnet_model.load_state_dict(torch.load('resnet50_classifier.pth', map_location=device))
    resnet_model.eval()
    
    # Load EfficientNet model
    efficientnet_model = EfficientNetClassifier(num_classes=2)
    efficientnet_model.load_state_dict(torch.load('efficientnet_classifier.pth', map_location=device))
    efficientnet_model.eval()
    
    # Load YOLO model
    yolo_model = YOLO('best_yolo_classifier.pt')
    
    # Example inference
    test_image_path = "path/to/test/image.jpg"
    
    if os.path.exists(test_image_path):
        # Preprocess image for PyTorch models
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        image = Image.open(test_image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # ResNet inference
        with torch.no_grad():
            resnet_output = resnet_model(input_tensor)
            resnet_pred = torch.softmax(resnet_output, dim=1)
        
        # EfficientNet inference
        with torch.no_grad():
            eff_output = efficientnet_model(input_tensor)
            eff_pred = torch.softmax(eff_output, dim=1)
        
        # YOLO inference
        yolo_results = yolo_model(test_image_path)
        
        print("Inference Results:")
        print(f"ResNet50 predictions: {resnet_pred}")
        print(f"EfficientNet predictions: {eff_pred}")
        print(f"YOLO results: {yolo_results}")

if __name__ == "__main__":
    main()
    # Uncomment to run inference example
    # load_and_inference_example()ample()

Data directory not found. Creating sample data...
Sample data created in 'sample_data' directory
Using device: cuda
Classes: ['folderA', 'folderB']
Train samples: 100
Val samples: 40
TRAINING MODEL 1: ResNet50


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/vibhanshu92/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%



Training ResNet50...
Epoch 1/10, Batch 0/4, Loss: 0.6877
Epoch 1/10:
  Train Loss: 0.5910
  Val Loss: 0.0673
  Val Accuracy: 1.0000
  New best model saved! Accuracy: 1.0000
--------------------------------------------------
Epoch 2/10, Batch 0/4, Loss: 0.0062
Epoch 2/10:
  Train Loss: 0.0644
  Val Loss: 0.0115
  Val Accuracy: 1.0000
--------------------------------------------------
Epoch 3/10, Batch 0/4, Loss: 0.0068
Epoch 3/10:
  Train Loss: 0.0067
  Val Loss: 0.0001
  Val Accuracy: 1.0000
--------------------------------------------------
Epoch 4/10, Batch 0/4, Loss: 0.0017
Epoch 4/10:
  Train Loss: 0.0013
  Val Loss: 0.0000
  Val Accuracy: 1.0000
--------------------------------------------------
Epoch 5/10, Batch 0/4, Loss: 0.0005
Epoch 5/10:
  Train Loss: 0.0009
  Val Loss: 0.0000
  Val Accuracy: 1.0000
--------------------------------------------------
Epoch 6/10, Batch 0/4, Loss: 0.0002
Epoch 6/10:
  Train Loss: 0.0058
  Val Loss: 0.0000
  Val Accuracy: 1.0000
----------------

Ultralytics 8.3.174 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 3708MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=sample_data, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

train: Scanning /home/vibhanshu92/Downloads/Task_1_Image_Classification/sample_data/train... 100 images, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 4369.93it/s]

...

      30/30      1.11G     0.4133          4        224: 100%|██████████| 7/7 [00:00<00:00, 28.88it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00, 76.41it/s]

                   all        0.5          1



30 epochs completed in 0.004 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 3708MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/vibhanshu92/Downloads/Task_1_Image_Classification/sample_data/train... found 100 images in 2 classes ✅ 
val: /home/vibhanshu92/Downloads/Task_1_Image_Classification/sample_data/valid... found 40 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00, 23.02it/s]


                   all          1          1
Speed: 0.9ms preprocess, 1.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
INFERENCE SPEED BENCHMARKING
ResNet50 - Average inference time: 9.14ms ± 1.31ms
EfficientNet - Average inference time: 12.16ms ± 1.30ms

image 1/1 /home/vibhanshu92/Downloads/Task_1_Image_Classification/sample_data/valid/folderA/image_000.jpg: 224x224 folderA 0.63, folderB 0.37, 3.8ms
Speed: 8.6ms preprocess, 3.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/vibhanshu92/Downloads/Task_1_Image_Classification/sample_data/valid/folderA/image_000.jpg: 224x224 folderA 0.63, folderB 0.37, 4.8ms
Speed: 2.3ms preprocess, 4.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/vibhanshu92/Downloads/Task_1_Image_Classification/sample_data/valid/folderA/image_000.jpg: 224x224 folderA 0.63, folderB 0.37, 5.3ms
Speed: 1.4ms preprocess, 5.3ms inference, 0.1ms postproces